In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
optimisation_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
optimisation_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
#Look at  STATUS value_counts
optimisation_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
optimisation_clean=optimisation_df.drop(['EIN','NAME','STATUS','SPECIAL_CONSIDERATIONS'],axis=1)
optimisation_clean

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts=optimisation_clean['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = list(application_counts[application_counts < 500].index)
# Replace in dataframe
for app in application_types_to_replace:
    optimisation_clean['APPLICATION_TYPE'] = optimisation_clean['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
optimisation_clean['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts=optimisation_clean['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 200].index)

# Replace in dataframe
for cls in classifications_to_replace:
    optimisation_clean['CLASSIFICATION'] = optimisation_clean['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
optimisation_clean['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate categorical variable lists
application_cat = optimisation_clean.dtypes[optimisation_clean.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies= pd.get_dummies(optimisation_clean)
application_dummies

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,5000,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,5000,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,5000,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [10]:
# Split our preprocessed data into our features and target arrays
y=application_dummies.IS_SUCCESSFUL.values
X=application_dummies.drop(["IS_SUCCESSFUL"], axis='columns').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
## MODEL 1-increase in neurons and epochs
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  20
hidden_nodes_layer2 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                860       
                                                                 
 dense_1 (Dense)             (None, 20)                420       
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1301 (5.08 KB)
Trainable params: 1301 (5.08 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the Sequential model together and customise metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 7s 5ms/step - loss: 0.5833 - accuracy: 0.7105
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5556 - accuracy: 0.7300
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5523 - accuracy: 0.7302
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5507 - accuracy: 0.7326
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5491 - accuracy: 0.7331
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5479 - accuracy: 0.7318
Epoch 7/100
804/804 [==============================] - 4s 6ms/step - loss: 0.5467 - accuracy: 0.7341
Epoch 8/100
804/804 [==============================] - 7s 8ms/step - loss: 0.5464 - accuracy: 0.7324
Epoch 9/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5459 - accuracy: 0.7345
Epoch 10/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5455 - accura

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 3s - loss: 0.5522 - accuracy: 0.7264 - 3s/epoch - 12ms/step
Loss: 0.5521538257598877, Accuracy: 0.7264139652252197


In [16]:
## MODEL 2-increase in hidden layers and epochs
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  20
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 20)                860       
                                                                 
 dense_4 (Dense)             (None, 20)                420       
                                                                 
 dense_5 (Dense)             (None, 10)                210       
                                                                 
 dense_6 (Dense)             (None, 10)                110       
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1611 (6.29 KB)
Trainable params: 1611 (6.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Compile the Sequential model together and customise metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 8s 5ms/step - loss: 0.5823 - accuracy: 0.7124
Epoch 2/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5542 - accuracy: 0.7286
Epoch 3/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5517 - accuracy: 0.7313
Epoch 4/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5492 - accuracy: 0.7321
Epoch 5/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5488 - accuracy: 0.7324
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5479 - accuracy: 0.7333
Epoch 7/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5465 - accuracy: 0.7341
Epoch 8/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5462 - accuracy: 0.7344
Epoch 9/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5458 - accuracy: 0.7338
Epoch 10/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5453 - accura

804/804 [==============================] - 3s 4ms/step - loss: 0.5348 - accuracy: 0.7409
Epoch 161/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5349 - accuracy: 0.7402
Epoch 162/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5343 - accuracy: 0.7409
Epoch 163/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5349 - accuracy: 0.7404
Epoch 164/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5346 - accuracy: 0.7414
Epoch 165/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5348 - accuracy: 0.7403
Epoch 166/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5343 - accuracy: 0.7409
Epoch 167/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5343 - accuracy: 0.7405
Epoch 168/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5346 - accuracy: 0.7409
Epoch 169/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5347 - a

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5606 - accuracy: 0.7266 - 1s/epoch - 4ms/step
Loss: 0.5605924725532532, Accuracy: 0.7266472578048706


In [20]:
## MODEL 3-change of activation function
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  20
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 16


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="leaky_relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="leaky_relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 20)                860       
                                                                 
 dense_9 (Dense)             (None, 20)                420       
                                                                 
 dense_10 (Dense)            (None, 16)                336       
                                                                 
 dense_11 (Dense)            (None, 16)                272       
                                                                 
 dense_12 (Dense)            (None, 1)                 17        
                                                                 
Total params: 1905 (7.44 KB)
Trainable params: 1905 (7.44 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
# Compile the Sequential model together and customise metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 6s 5ms/step - loss: 0.7639 - accuracy: 0.6882
Epoch 2/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5713 - accuracy: 0.7243
Epoch 3/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5637 - accuracy: 0.7294
Epoch 4/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5602 - accuracy: 0.7316
Epoch 5/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5550 - accuracy: 0.7306
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5699 - accuracy: 0.7212
Epoch 7/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5536 - accuracy: 0.7301
Epoch 8/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5502 - accuracy: 0.7318
Epoch 9/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5500 - accuracy: 0.7328
Epoch 10/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5523 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimisation.h5')

# Overview

The purpose of this analysis was to use historical data from organizations over the years that have received funding from Alphabet soup and create a binary classification model that can predict if an Alphabet Soup-funded organisation will be successful based on the features used in the dataset. 


The features used from the historical data consisted of the following features:APPLICATION_TYPE(Alphabet Soup application type), AFFILIATION (Affiliated sector of industry), CLASSIFICATION(Government organisation classification), USE_CASE(Use case for funding), ORGANIZATION(Organisation type), STATUS(Active status; only for the inital model), INCOME_AMT(Income classification), SPECIAL_CONSIDERATIONS(Special considerations for application), ASK_AMT(Funding amount requested) and IS_SUCCESSFUL (Was the money used effectively).


The stages of designing my deep learning model and creating my binary classification model, consisted of:

-Preprocessing:cleaning the data and splitting it into training and testing sets

-Compiling: defining and designning the classification model 

-Training: using the classification model on the training data set

-Evaluating : evaluating the performance of the model using the test data set

-Optimising: optimising the original model using different methods






# Results

## Data Preprocessing

The target of my model was the IS_SUCCESSFUL variable seeing as it deemed whether an organization used their fund effectively and in my opinion the most important factor in terms of predicting whether an organization will be successful if funded.

The features of my model consisted of the variables I previously stated in my overview and the reason for that being was that I thought all were beneficial in regards to helping me with my prediction. 

The variables that I did remove from the original dataset were EIN and NAME seeing as they were only used for identification purposes and I also removed STATUS when attempting to optimise my model seeing as the data for that variable was significatnly skewed (1=34294 and 0=5) and it could possibly increase the accuracy of the model if removed.

## Compiling, Training, and Evaluating the Model

Initially with my original model I went with an overall conservative approach just to see where the accuracy was at, consisting of: only using 2 hidden layers, 8 neurons in each, relu as the activation function for the hidden layers and sigmoid for the out put layer.

Following the initial model I created another 3 models each with a  different slight adjustment of a hyperparameter compared to the inital model in order to make my model more effective. In model 1 in increased the neurons to 20 in  each hidden layer and also increased the epochs from 50 to 100 with a slight increase in accuracy. In model 2 I increased the hidden layers from 2 to 4 and the epochs again from 100 to 200 (from the previous model-model 1) with a slight decrease in accuracry from the previous model.Finally, in model 3 I changed the activation functions of my hidden layers(hidden layer 2 and 3 from relu to LeakyRelu) and my output layer (from sigmoid to tanh) and in comparison to all the models I created that reached the highest accuracy of 0.7269970774650574. Nonetheless, I was not able to achieve the target model performance of 75%.

# Summary

Overall with all the models built and all the adjustments made to each of the hyperparameters in each model (neurons, hidden layers and activation function) the models were unsuccessful in reaching a target predictive accuracy higher than 75%. With each model and the gradual adjustments of each hyperparameter from initially increasing the neurons and the epochs to increasing the hidden layers and the epochs once again to finally altering the activation functions there was some gradual increas in the accuracy of the model. Therefore, adjustments to the hyperparameters do make a difference to the accuracy of the model, but in this case only minimally, in conjunction, with the preprocessing of the data.

In retrospect, a deep learning model could achieve the desired predictive accuracy of 75% in my opinion seeing as it is a large  and complex dataset being used.The specifics of the deep learning model would require much more pre-processing of the data in comparison to my models(e.g. removal of more variables and bucketing not only categorical but numerical values), and also a much more less conservative approach to the adjustments of the hyperparameters in comparison to my models (e.g. significant increase in number of neurons and epochs and possible a few more hidden layers). Furthermore, it could also be used in conjunction with the kerastuner library in order to assist in identifying the best choice of model by deciding the best combination of hyperparameters(neurons, hidden layers and activation functions). 